In [2]:
#
# classifer notebook
#

# for Colab paths
# import sys
# sys.path.append('/content/')
#
#!nvidia-smi
#!nvidia-smi -q


import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import Lambda, Compose
from LSTMLandmarkDataset import LSTMLandmarkDataset
from torch.utils.data.sampler import SubsetRandomSampler

torch.set_default_dtype(torch.float64)

#
# MODEL
#
# Should be 63 classes:
# Price B/S * 10                 -> 20
# QTY B/S * 10                   -> 20
# QTY B/S 10,20-100              -> 20
# Action Cancel, Market, Garbage ->  3
#                                -> 63
class NN(torch.nn.Module):
    
    def __init__(self, input_size, hidden_size, num_classes):
        super(NN, self).__init__()
        self.fc1 = torch.nn.Linear(input_size, hidden_size)
        self.relu = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(hidden_size, num_classes)
        #self.fc2 = torch.nn.Linear(hidden_size, 48)
        #self.fc3 = torch.nn.Linear(48, num_classes)
        
        #44 with dropout .025
        #28 with .01
        self.dropout = torch.nn.Dropout(p=0.01)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        #out = self.dropout(out)
        
        out = self.fc2(out)
        #out = self.relu(out)
        #out = self.dropout(out)
        
        #out = self.fc3(out)
        
        return out

    
#
# PARAMS
#

batch_size = 12
hidden_size = 96
learning_rate = .01
num_epochs = 250


#
# INITIAL DATA
#

# for directory load each file
# generate mapping of file -> class -> idx
# if available copy data to gpu (model set below)
# typically need model + tensors (label and value) moved over. 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

transformations = Compose([
    Lambda(lambda x: torch.tensor(x.values).to(device))
])
target_transformations = Compose([
    Lambda(lambda x: torch.tensor(x).to(device))
])

dataset = LSTMLandmarkDataset("/home/jovyan/train/lstm_data",
                              "/home/jovyan/model",
                              transform=transformations)

num_classes = dataset.num_class
input_size = dataset.input_size() #2 * (21 * 3) + 12 + 1 + 10 #149 + 1 label = 150

training_indices, validation_indices = dataset.train_validation_indices(split_p = .2)
        
train_sampler = SubsetRandomSampler(training_indices)
valid_sampler = SubsetRandomSampler(validation_indices)
    
train_dataloader = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
valid_dataloader = DataLoader(dataset, batch_size=batch_size, sampler=valid_sampler)


#
# INIT MODELS, LOSS FN, GRAD
#

model = NN(input_size, hidden_size, num_classes).to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  

#
# TRAIN
#

running_loss_epoch = 0
for epoch in range(num_epochs):
    for batch_idx, (labels, landmarks) in enumerate(train_dataloader):  

        # zero out accumulated gradients
        optimizer.zero_grad()
        
        # forward pass
        outputs = model(landmarks)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        loss.backward()
        optimizer.step()
        
        # NB: len(dataloader) is num of batches
        running_loss_epoch += loss.item()
        
        if ((epoch+1) % 10 == 0) and ((batch_idx+1) % len(train_dataloader) == 0):
            print ('Epoch [{}/{}], Loss per (random) batch: {:.4f}, Running Loss (per N epoch): {:.4f}' 
                   .format(epoch+1, num_epochs, loss.item(), running_loss_epoch))
            running_loss_epoch = 0

    

Epoch [10/250], Loss per (random) batch: 0.0015, Running Loss (per N epoch): 64.1986
Epoch [20/250], Loss per (random) batch: 0.0011, Running Loss (per N epoch): 15.3650
Epoch [30/250], Loss per (random) batch: 0.8816, Running Loss (per N epoch): 7.8740
Epoch [40/250], Loss per (random) batch: 0.0002, Running Loss (per N epoch): 6.3220
Epoch [50/250], Loss per (random) batch: 0.0001, Running Loss (per N epoch): 5.1563
Epoch [60/250], Loss per (random) batch: 0.0008, Running Loss (per N epoch): 5.0350
Epoch [70/250], Loss per (random) batch: 0.0010, Running Loss (per N epoch): 4.1451
Epoch [80/250], Loss per (random) batch: 0.0007, Running Loss (per N epoch): 4.6147
Epoch [90/250], Loss per (random) batch: 0.0004, Running Loss (per N epoch): 4.6884
Epoch [100/250], Loss per (random) batch: 0.0011, Running Loss (per N epoch): 3.3112
Epoch [110/250], Loss per (random) batch: 0.0000, Running Loss (per N epoch): 4.3031
Epoch [120/250], Loss per (random) batch: 0.0000, Running Loss (per N ep

In [3]:
#
# EXPORT
#
# NB: LandmarkDataset.py updates meta.json with class index
#

#dummy_input = torch.zeros(input_size)
#model.load_state_dict(torch.load('./model_overfit.pt'))
#torch.onnx.export(model, dummy_input, 'onnx_model.onnx', export_params=True,
#                  input_names = ['landmarks'], output_names = ['class'], verbose=True)

#import onnx
#onnx_model = onnx.load("./onnx_model.onnx")
#onnx.checker.check_model(onnx_model)


In [4]:
#
# EVAL
#
torch.set_printoptions(precision=4, sci_mode=False)
softmax = torch.nn.Softmax(dim=1)
accuracy = 0
count = 0
with torch.no_grad():
    
    matches = {}
    counts = {}
    
    for batch_idx, (labels, landmarks) in enumerate(valid_dataloader):  
        out = model(landmarks)
        prob = softmax(out.data)     #setup for threshold or 'garbage' class
        _, klass = torch.max(out.data, 1)

        #print(klass, labels, klass==labels)
        #print(prob)
        #print("-----")

        # x is class
        for i, x in enumerate(klass.tolist()):
            label = int(labels[i])
            if x == labels[i]:
                matches[x] = matches.get(x, 0) + 1
            else:
                # track label-side class as well
                matches[x] = matches.get(x, 0)
                matches[ label ] = matches.get(label, 0)
                                
            counts[label] = counts.get(label, 0) + 1
            counts[x] = counts.get(x, 0) + 1
            
        # aggregate accuracy
        accuracy += (klass == labels).sum().item()
        count += len(labels)
        

print('--------')
print("Accuracy {}/{} : {:.4f}".format(accuracy, count, accuracy/count))
print("------------")

# class vs percentage label match - track whether some gestures have bad data
acc = sorted([(int(k), v / counts.get(k, 1)) for k,v in matches.items()], key=lambda x:x[1])
for klass, match in acc:
    print(klass, " {:.4f}".format(match))


--------
Accuracy 121/121 : 1.0000
------------
1  0.5000
2  0.5000
0  0.5000
